In [1]:
### Import libraries

import pandas as pd
import igraph as ig
import auxiliar_path
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
### Global variables
### Global variables

DATASET = "AMZ" # AMZ, HC, PM, UN, TOY
NODE_TYPE = True

PATH_DATASET = auxiliar_path.get_path_dataset(DATASET)
PATH_NODETYPE = auxiliar_path.get_path_topbot(NODE_TYPE)

GLOBAL_PATH = "/Users/ddiaz/Documents/code/phd-thesis-lab/"

# File CSV
### GLOBAL variables

GRAPH_PATH = "/Users/ddiaz/Documents/code/phd-thesis-lab/12-third_year/00-Data/01-AMZ/02-Graphs/01-Top/AMZ_top_resall_DF_alpha2.graphml"
#GRAPH_PATH = "/Users/ddiaz/Documents/code/phd-thesis-lab/12-third_year/00-Data/02-HC/02-Graphs/02-Bot/HC_bot_weights_DF_alpha05.graphml"
GRAPH_PATH = "../"

In [3]:
### Read GRAPH

g = ig.read(GRAPH_PATH)
print(g.summary(), "\n")

print("Graph info:")
print("\t|V| =", g.vcount())
print("\t|E| =", g.ecount())
print("\t d  =", g.density())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ddiaz/Documents/code/phd-thesis-lab/12-third_year/00-Data/01-AMZ/02-Graphs/01-Top/AMZ_top_resall_DF_alpha2.graphml'

In [7]:
### Community Detection

comms = g.community_multilevel(weights=g.es["weight"])
g.vs["cls"] = comms.membership
print(comms.summary(), "\n")

Clustering with 9560 elements and 423 clusters 



In [45]:
comms_with_one = {subgraph.vs["cls"][0]: subgraph for i, subgraph in enumerate(comms.subgraphs()) if len(subgraph.vs) > 5}
comms_with_one

{0: <igraph.Graph at 0x132d7e840>,
 1: <igraph.Graph at 0x132d7dc40>,
 2: <igraph.Graph at 0x132d7dd40>,
 3: <igraph.Graph at 0x132d7e740>,
 4: <igraph.Graph at 0x132d7eb40>,
 5: <igraph.Graph at 0x132d7e340>,
 6: <igraph.Graph at 0x132d7ec40>,
 7: <igraph.Graph at 0x132d7ee40>,
 8: <igraph.Graph at 0x132d7f040>,
 9: <igraph.Graph at 0x132d7f140>,
 10: <igraph.Graph at 0x132d7f240>,
 11: <igraph.Graph at 0x132d7f340>,
 12: <igraph.Graph at 0x132d7f440>,
 13: <igraph.Graph at 0x132d7f540>,
 14: <igraph.Graph at 0x132d7f640>,
 15: <igraph.Graph at 0x132d7f840>,
 16: <igraph.Graph at 0x132d7f940>,
 17: <igraph.Graph at 0x132d7fa40>,
 20: <igraph.Graph at 0x132d7cb40>,
 21: <igraph.Graph at 0x132d7cd40>,
 22: <igraph.Graph at 0x132d7ca40>,
 23: <igraph.Graph at 0x132d7cf40>,
 24: <igraph.Graph at 0x132d7d040>,
 25: <igraph.Graph at 0x132d7d240>,
 27: <igraph.Graph at 0x132d7d440>,
 30: <igraph.Graph at 0x132d7d840>,
 35: <igraph.Graph at 0x132d7de40>,
 37: <igraph.Graph at 0x132d7e040>,
 4

In [26]:
def internal_external_degree_node(node):
    """Compute the internal degree k_i^int of node i in a community C."""
    node_neighs = node.neighbors()
    node_community = node["cls"]

    internal_degree = [1 for i in node_neighs if i["cls"]==node_community]
    external_degree = [1 for i in node_neighs if i["cls"]!=node_community]
    
    return sum(internal_degree), sum(external_degree)

def check_strong_community(graph, communities):
    """Returns if it is a strong community."""

    dict_to_ret = {}

    for C in communities.keys():
        

In [27]:
internal_external_degree_node(test_node)

(10, 9)

In [19]:
neis

[igraph.Vertex(<igraph.Graph object at 0x132c70040>, 23, {'id': 23.0, 'type': 0.0, 'freq': 24.0, 'cls': 0}),
 igraph.Vertex(<igraph.Graph object at 0x132c70040>, 247, {'id': 247.0, 'type': 0.0, 'freq': 10.0, 'cls': 0}),
 igraph.Vertex(<igraph.Graph object at 0x132c70040>, 254, {'id': 254.0, 'type': 0.0, 'freq': 24.0, 'cls': 0}),
 igraph.Vertex(<igraph.Graph object at 0x132c70040>, 366, {'id': 366.0, 'type': 0.0, 'freq': 16.0, 'cls': 5}),
 igraph.Vertex(<igraph.Graph object at 0x132c70040>, 1190, {'id': 1190.0, 'type': 0.0, 'freq': 18.0, 'cls': 0}),
 igraph.Vertex(<igraph.Graph object at 0x132c70040>, 1644, {'id': 1644.0, 'type': 0.0, 'freq': 16.0, 'cls': 5}),
 igraph.Vertex(<igraph.Graph object at 0x132c70040>, 1705, {'id': 1705.0, 'type': 0.0, 'freq': 10.0, 'cls': 0}),
 igraph.Vertex(<igraph.Graph object at 0x132c70040>, 1896, {'id': 1896.0, 'type': 0.0, 'freq': 6.0, 'cls': 0}),
 igraph.Vertex(<igraph.Graph object at 0x132c70040>, 1958, {'id': 1958.0, 'type': 0.0, 'freq': 4.0, 'cls': 